In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

data = pd.read_csv('/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/artifacts/data/europarl/processed/bg_en/bg_en.csv')
data.head()

,source_text,target_text
0,4.,4.
1,Ратифициране и прилагане на актуализирани конв...,The ratification and implementation of the upd...
2,"Действия, предприети вследствие резолюции на П...",Action taken on Parliament's resolutions: see ...
3,Двустранни споразумения между държави-членки и...,Bilateral agreements between Member States and...
4,Следващата точка е общо разискване по,The next item is the joint debate on


In [2]:
import pandas as pd
import numpy as np

def save_csv_percentage(input_file, output_file, percentage):
    df = pd.read_csv(input_file)
    
    num_rows = len(df)
    num_rows_to_save = int((percentage / 100) * num_rows)
    
    df_sampled = df.sample(n=num_rows_to_save, random_state=42)
    
    df_sampled.to_csv(output_file, index=False)
    
    print(f"Saved {num_rows_to_save} rows to {output_file}")

# Example usage
input_csv_path = '/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/artifacts/data/europarl/processed/bg_en/bg_en.csv'
output_csv_path = '/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/artifacts/data/europarl/processed/bg_en/bg_en_new.csv'
percentage_to_save = 50.0 

save_csv_percentage(input_csv_path, output_csv_path, percentage_to_save)

Saved 101963 rows to /Users/vikkyfury/Desktop/pr/program/LangWiz-AI/artifacts/data/europarl/processed/bg_en/bg_en_new.csv


In [3]:
data = pd.read_csv('/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/artifacts/data/europarl/processed/bg_en/bg_en_new.csv')
data.head()

,source_text,target_text
0,"Бих искал да кажа, че като новак в тази роля о...","I would like to say that, as a newcomer to thi..."
1,"Увеличението, което иска Парламента, не е прищ...",The increase that Parliament is requesting is ...
2,Грузия в кратък срок постигна много.,Georgia has achieved much in a short time.
3,Подобни слепи методи току-що показаха ограниче...,These blind methods have just demonstrated the...
4,Преработеното предложение взема предвид резулт...,The revised proposal takes into account the ou...


In [7]:
train_data, val_data = train_test_split(data, test_size=0.2)


In [8]:
val_data = val_data[:2000]

In [9]:
from datasets import Dataset, DatasetDict
train_dataset = Dataset.from_pandas(train_data)
val_dataset = Dataset.from_pandas(val_data)

dataset = DatasetDict({
    'train': train_dataset,
    'val': val_dataset
})
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['source_text', 'target_text', '__index_level_0__'],
        num_rows: 81570
    })
    val: Dataset({
        features: ['source_text', 'target_text', '__index_level_0__'],
        num_rows: 2000
    })
})


In [10]:
from transformers import MarianTokenizer
model_token = 'Helsinki-NLP/opus-mt-bg-en'
tokenizer = MarianTokenizer.from_pretrained(model_token,return_tensors = 'pt')

In [11]:
with tokenizer.as_target_tokenizer():
        encodings = tokenizer('hi bub', max_length = 128,
                                         truncation=True, padding=True,
                                         return_tensors = 'pt')
        
encodings

/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/.venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:4144: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


{'input_ids': tensor([[ 5061, 16661,   353,     0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [12]:
tokenizer(text_target='hi bub', max_length = 128,
                                         truncation=True, padding=True,
                                         return_tensors = 'pt')

{'input_ids': tensor([[ 5061, 16661,   353,     0]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [13]:
max_length=128

def preprocess_function(example):
        # print(example)
        try:
            inputs = example['source_text']
            targets = example['target_text']
            
            if not isinstance(inputs,(str,list)):
                inputs = str(inputs)
            if not isinstance(targets,(str,list)):
                targets = str(targets)
            
            input_encodings = tokenizer(inputs, max_length = max_length,
                                             truncation=True, padding=True,
                                             return_tensors = 'pt')#.to(device)
            
            target_encodings= tokenizer(max_length = max_length,
                                             truncation=True, padding=True,
                                             return_tensors = 'pt', text_target=targets)#.to(device)
            
            return {
                'input_ids' : input_encodings['input_ids'],
                'attention_mask' : input_encodings['attention_mask'],
                'labels': target_encodings['input_ids']       
            }
        except Exception as e:
            print('---------------------------------')
            print(e)
            print(example)
            print(type(example['source_text']), type(example['target_text']))
            
            

In [14]:
from datasets import load_from_disk

train_data = dataset['train'].map(preprocess_function, batched=True)
train_data.save_to_disk('./bg_train_data')


Map:   0%|          | 0/81570 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/81570 [00:00<?, ? examples/s]

In [15]:
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq
model_bg_en = AutoModelForSeq2SeqLM.from_pretrained(model_token)

In [16]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer,model = model_bg_en)
train_data = load_from_disk('./bg_train_data')




In [17]:
from transformers import TrainingArguments, Trainer

In [18]:
model = model_bg_en

In [19]:
from tqdm import tqdm
trainer_args = TrainingArguments(
            output_dir='.output',
            save_strategy='epoch',
            num_train_epochs = 3,
            learning_rate = 0.00002,
            per_device_train_batch_size = 32,
            per_device_eval_batch_size= 32,
            weight_decay= 0.02,
            save_total_limit = 3
        )

trainer = Trainer(
            model = model, args = trainer_args,
            tokenizer = tokenizer, data_collator = seq2seq_data_collator,
            train_dataset = train_data,      
        )
for param in model.model.encoder.parameters():
    param.requires_grad = False
# Unfreeze the decoder layers and the final linear layer
for param in model.model.decoder.parameters():
    param.requires_grad = True
for param in model.lm_head.parameters():
    param.requires_grad = True

trainer.train()

Step,Training Loss
500,0.315000
1000,0.242600
1500,0.237000
2000,0.237100
2500,0.236500
3000,0.224300
3500,0.219000
4000,0.218900
4500,0.218500
5000,0.220300


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61812]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61812]], 'forced_eos_token_id': 0}
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strate

TrainOutput(global_step=7650, training_loss=0.22808338140350543, metrics={'train_runtime': 6750.5134, 'train_samples_per_second': 36.251, 'train_steps_per_second': 1.133, 'total_flos': 8295268881530880.0, 'train_loss': 0.22808338140350543, 'epoch': 3.0})

In [20]:
trainer.save_model(output_dir = './bg_en_model')
tokenizer.save_pretrained('./bg_en_model_tokenizer')

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[61812]], 'forced_eos_token_id': 0}


('./bg_en_model_tokenizer/tokenizer_config.json',
 './bg_en_model_tokenizer/special_tokens_map.json',
 './bg_en_model_tokenizer/vocab.json',
 './bg_en_model_tokenizer/source.spm',
 './bg_en_model_tokenizer/target.spm',
 './bg_en_model_tokenizer/added_tokens.json')

In [21]:
from transformers import *
import torch
model = AutoModelForSeq2SeqLM.from_pretrained('./bg_en_model')
tokenizer = AutoTokenizer.from_pretrained('./bg_en_model_tokenizer')

/Users/vikkyfury/Desktop/pr/program/LangWiz-AI/.venv/lib/python3.12/site-packages/transformers/deepspeed.py:24: FutureWarning: transformers.deepspeed module is deprecated and will be removed in a future version. Please import deepspeed modules directly from transformers.integrations
  warnings.warn(
loading configuration file ./bg_en_model/config.json
Model config MarianConfig {
  "_name_or_path": "./bg_en_model",
  "_num_labels": 3,
  "activation_dropout": 0.0,
  "activation_function": "swish",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "MarianMTModel"
  ],
  "attention_dropout": 0.0,
  "bad_words_ids": [
    [
      61812
    ]
  ],
  "bos_token_id": 0,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 512,
  "decoder_attention_heads": 8,
  "decoder_ffn_dim": 2048,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 61812,
  "decoder_vocab_size": 61813,
  "dropout": 0.1,
  "encoder_attention_heads

In [22]:
def tokenize_function(example):
    try:
        inputs = example['source_text']
        targets = example['target_text']
        
        if not isinstance(inputs,(str,list)):
            inputs = str(inputs)
        if not isinstance(targets,(str,list)):
            targets = str(targets)
                
        return tokenizer(example['source_text'], padding='max_length', max_length=128, truncation=True)
    except Exception as e:
            print(e)
    
tokenized_dataset = dataset['val'].map(tokenize_function, batched=True)
    

input_ids = torch.tensor(tokenized_dataset['input_ids'])
attention_mask = torch.tensor(tokenized_dataset['attention_mask'])


Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [23]:
model.eval()
predictions = []
with torch.no_grad():
    for input_id, attention_msk in zip(input_ids, attention_mask):
        output = model.generate(input_ids=input_id.unsqueeze(0), attention_mask=attention_msk.unsqueeze(0))
        prediction = tokenizer.decode(output[0], skip_special_tokens=True)
        predictions.append(prediction)


In [27]:
predictions

['The EU must not stand by; the EU must do more to protect women and children, as well as the rule of law for everyone in Egypt and Libya.',
 '- Report: Paulo Casaca',
 "The European Network and Information Security Agency plays a special role in supporting Member States' efforts to cooperate in this area at EU level.",
 'Approval of the minutes of the previous sitting: see Minutes',
 "At this stage, we simply want Mr Lewandowski to be aware of what Parliament's budgetary priorities are, namely, our proposals.",
 'Our priorities include the need to strengthen the role of the WTO in monitoring and analysing protectionism; accession to the WTO - such as the accession of the least developed countries to the WTO, which we certainly believe should be encouraged; increasing the number of regional free trade agreements, for example - here we must ensure that they actually complement the multilateral trade system; and finally, the contribution of trade policy to combating climate change.',
 'T

In [24]:
val_texts = val_data['target_text'].tolist()
len(val_texts)

2000

In [25]:
import evaluate
actual = val_texts
predicted = predictions
bleu = evaluate.load('bleu')
bleu_score = bleu.compute(predictions = predicted,references = actual)
print(f'BLEU Score: {bleu_score}')

BLEU Score: {'bleu': 0.48862381885787987, 'precisions': [0.7576770538243626, 0.5522669283611383, 0.42661899897854955, 0.33427719821162444], 'brevity_penalty': 0.9886204829191819, 'length_ratio': 0.9886847412054671, 'translation_length': 52950, 'reference_length': 53556}


In [26]:
sacrebleu = evaluate.load('sacrebleu')
sacrebleu_score = sacrebleu.compute(predictions = predicted,references = actual)
print(f'SACREBLEU Score: {sacrebleu_score}')

SACREBLEU Score: {'score': 48.862381885787975, 'counts': [40119, 28138, 20883, 15701], 'totals': [52950, 50950, 48950, 46970], 'precisions': [75.76770538243626, 55.226692836113834, 42.66189989785495, 33.42771982116244], 'bp': 0.9886204829191821, 'sys_len': 52950, 'ref_len': 53556}


## Configured till above